In [384]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import cvxpy as cp
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import matplotlib.pyplot as plt

abspath = os.path.abspath(os.getcwd())
finpath = Path(abspath).resolve().parent
parent_path = str(finpath) + '/'

final_df = pd.read_csv(parent_path + "FinalDF.csv")
adj_counties = pd.read_csv(parent_path + "AdjacentDelta.csv")
final_df["Population"] = np.sqrt(final_df["Population"]) / np.sqrt(final_df["Population"].max()) 

In [2]:
def standardize_df(df):
    '''
    Standardizes our dataframes, which is essential for proper implementation of the LASSO.
    '''
    df = df.select_dtypes(exclude=['object'])
    feature_cols = df.columns[2:]
    std_final_df = pd.DataFrame(StandardScaler().fit_transform(df.iloc[:, 2:]), columns=feature_cols)
    pop_df = df["Population"]
    return std_final_df, pop_df

In [273]:
median_income = final_df["Median Household Income"].median()
df_upper_income = final_df[final_df["Median Household Income"] >= median_income]
df_lower_income = final_df[final_df["Median Household Income"] < median_income]

# Here, our dataframes are not the same shapes because % Completed High School has many values at the median,
# but the number of observations in the dataframes are 1197 for the upper bracket and 1209 for the lower bracket,
# which we found to be acceptable for running our regression analysis on
median_edu = final_df["% Completed High School"].median()
df_upper_edu = final_df[final_df["% Completed High School"] > median_edu]
df_lower_edu = final_df[final_df["% Completed High School"] <= median_edu]

# Adjacent counties regression
adj_counties = adj_counties.select_dtypes(exclude=['object'])


# Standardizing values and creating all the matrices we need to run Lasso
df_lst = [final_df, df_upper_income, df_lower_income, df_upper_edu, df_lower_edu, adj_counties] 
params_lst = []
for frame in df_lst:
    std_frame, pop_df = standardize_df(frame)
    label_df = std_frame["% Smokers"]
    IV_df = std_frame["Cig Tax Rate"]
    features_df = std_frame.drop(columns=["% Smokers", "Cig Tax Rate"])
    p = np.array([pop_df]).T
    X = np.array(features_df)
    IV = np.array([IV_df]).T
    y = np.array([label_df]).T
    values = [X, IV, y, p]
    params_lst.append(values)

In [326]:
def lasso_IV(lambda_IV, params):
    X, y, IV, p = params
    model = Lasso(alpha=lambda_IV, max_iter=10000).fit(X, IV.ravel())
    betas_IV = np.array(model.coef_)
    return betas_IV

def lasso_y(lambda_y, params):
    X, y, IV, p = params
    model = Lasso(alpha=lambda_y, max_iter=10000).fit(X, y.ravel())
    betas_y = np.array(model.coef_)
    return betas_y

def OLS(betas_IV, betas_y, params):
    X, y, IV, p = params
    betas = abs(betas_y) + abs(betas_IV)
    betadict = {k: v for k, v in enumerate(betas.tolist())}
    relevantbetadict = {k:v for (k, v) in betadict.items() if v >= 10 ** -5}
    covariates = ((betas >= 10 ** -5)).flatten()
    relevantbetadict[48] = betas[0] 
    X_cov = np.concatenate((IV, X[:, covariates]), axis=1)
    model = sm.OLS(y, pd.DataFrame(X_cov, columns=[generated.columns.tolist()[2:][i] for i in relevantbetadict.keys()]))
    results = model.fit()
    betas = results.params
    relevantbetadict[48] = betas[0] 
    return betas, relevantbetadict, results

def double_lasso(lambda_IV, lambda_y, params):
    '''
    Runs the double-lasso procedure. 
    '''
    betas_IV = lasso_IV(lambda_IV, params)
    betas_y = lasso_y(lambda_y, params)
    betas, relevantbetadict, results = OLS(betas_IV, betas_y, params)


    return betas, relevantbetadict, results

def getlambdas():
    lambdaIV = []
    lambdaY = []
    for param in params_lst:
        CV = LassoCV(cv=10, max_iter=10000).fit(param[0], param[2].ravel())
        CV2 = LassoCV(cv=10, max_iter=10000).fit(param[0], param[1].ravel())

        lambdaY.append(CV.alpha_)
        lambdaIV.append(CV2.alpha_)

    return lambdaY, lambdaIV

lambdaY, lambdaIV = getlambdas()

In [327]:
'''
Obtains results for our six double-lasso regressions given that the list of correctly cross-validated
lambda hyperparameters have been generated.
'''
labels = pd.read_csv(parent_path + "\\CorrMatrix.csv").columns.tolist()[2:]

for index in range(len(params_lst)):
    beta, relevant, results = double_lasso(lambdaIV[index], lambdaY[index], params_lst[index])
    print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.481
Model:                            OLS   Adj. R-squared (uncentered):              0.471
Method:                 Least Squares   F-statistic:                              49.74
Date:                Tue, 31 May 2022   Prob (F-statistic):                   8.09e-299
Time:                        04:40:15   Log-Likelihood:                         -2625.1
No. Observations:                2406   AIC:                                      5338.
Df Residuals:                    2362   BIC:                                      5593.
Df Model:                          44                                                  
Covariance Type:            nonrobust                                                  
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------

In [479]:
# Testing accuracy of different types of models on generated data

def normmeans(frame):

    normalfits = []
    for column in frame.columns.tolist()[1:]:
        data = frame[column].to_numpy()

        mu, std = norm.fit(data)
        normalfits.append(mu)

    return normalfits

def DGP():
    frame = pd.read_csv(parent_path + "\\FinalDF.csv")
    frame.drop(columns=["State", "County", "Population"], inplace=True)
    means = np.array(normmeans(frame))
    cov = pd.read_csv(parent_path + "\\CovMatrix.csv").to_numpy()[:, 1:]

    data = np.random.multivariate_normal(means, cov, size=cov.shape[0])
    
    resultframe = pd.DataFrame(data, columns=frame.columns.tolist()[1:])
    # print(resultframe)

    return resultframe

# This can be put in a for loop to generate many samples, then call testmodel on each sample, record average error, 

generated = DGP()
scaler = StandardScaler()
generatedy = generated["% Smokers"].to_numpy().reshape(-1, 1)
generatedy = pd.DataFrame(scaler.fit_transform(generatedy), columns=["% Smokers"])
X = generated.drop(columns=["% Smokers"])
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns.tolist())

X_train, X_test, y_train, y_test = train_test_split(X[X.columns.tolist()[1:]], generatedy)

def trainmodel(xtrain, ytrain, modelname):
    if modelname == "OLS":
        model = sm.OLS(ytrain, xtrain)
        results = model.fit()

        return results.params.tolist()
    elif modelname == "Lasso":
        CV = LassoCV(cv=10, max_iter=10000).fit(xtrain, ytrain.ravel())
        # print(f"Lambda: {CV.alpha_}")
        model = Lasso(alpha=CV.alpha_, max_iter=10000).fit(xtrain, ytrain.ravel())
        betas = np.array(model.coef_)

        # print(betas)
        return betas
    elif modelname == "DLasso":
        IVtrain = xtrain.to_numpy()[:, -1]
        IVtrain.reshape(len(IVtrain), 1)
        CV = LassoCV(cv=10, max_iter=10000).fit(xtrain, IVtrain.ravel())
        model = Lasso(alpha=CV.alpha_, max_iter=10000).fit(xtrain, IVtrain.ravel())
        c1 = np.array(model.coef_)

        CV2 = LassoCV(cv=10, max_iter=10000).fit(xtrain, IVtrain.ravel())
        model = Lasso(alpha=CV.alpha_, max_iter=10000).fit(xtrain, ytrain.ravel())
        c2 = np.array(model.coef_)

        union = np.add(abs(c1), abs(c2))
        mask = (union < 10 ** -5)

        xtrain = np.array(xtrain)
        x_train_masked = []
        for xxx in xtrain:
            xxx[mask] = 0
            x_train_masked.append(xxx)
        X_cov = np.concatenate((np.vstack(IVtrain), np.array(x_train_masked)[:, 1:]), axis=1)

        model = sm.OLS(ytrain, X_cov)
        results = model.fit()

        return results.params.tolist()



def testmodel(xtest, ytest, xtrain, ytrain, modelname):
    beta = np.array(trainmodel(xtrain, ytrain, modelname)).reshape(-1, 1)

    y = np.array(ytest).reshape(-1, 1)
    X = np.array(xtest)

    error = y - X @ beta
    avgerror = np.average(abs(error))

    return avgerror

def getstats():
    OLSerr = []
    Lassoerr = []
    DLassoerr = []
    for _ in range(100):
        generated = DGP()
        scaler = StandardScaler()
        generatedy = generated["% Smokers"].to_numpy().reshape(-1, 1)
        generatedy = pd.DataFrame(scaler.fit_transform(generatedy), columns=["% Smokers"]).to_numpy()
        X = generated.drop(columns=["% Smokers"])
        X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns.tolist())
        X_train, X_test, y_train, y_test = train_test_split(X[X.columns.tolist()[1:]], generatedy)
        OLSerr.append(testmodel(X_test, y_test, X_train, y_train, "OLS"))
        Lassoerr.append(testmodel(X_test, y_test, X_train, y_train, "Lasso"))
        DLassoerr.append(testmodel(X_test, y_test, X_train, y_train, "DLasso"))
    
    totalavgOLS = np.average(abs(np.array(OLSerr)))
    totalavgLasso = np.average(abs(np.array(Lassoerr)))
    totalavgDLasso = np.average(abs(np.array(DLassoerr)))

    print(f"Average Error across all samples (OLS): {totalavgOLS}")
    print(f"Average Error across all samples (Lasso): {totalavgLasso}")
    print(f"Average Error across all samples (DLasso): {totalavgDLasso}")

getstats()






c:\Users\spont\anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: covariance is not positive-semidefinite.
c:\Users\spont\anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: covariance is not positive-semidefinite.


Average Error across all samples (OLS): 0.465250544418447
Average Error across all samples (Lasso): 0.3150280462798832
Average Error across all samples (DLasso): 0.3986487499997029
